## Importing from Keras

In [1]:
import keras

from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

if keras.backend.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [3]:
train_dir = '../data/Pandora18K_small_train_val_test_split/train'
valid_dir = '../data/Pandora18K_small_train_val_test_split/val'
test_dir  = '../data/Pandora18K_small_train_val_test_split/test'
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
valid_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen  = ImageDataGenerator(rescale=1. / 255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

valid_gen = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 14572 images belonging to 18 classes.
Found 1912 images belonging to 18 classes.
Found 1933 images belonging to 18 classes.


In [4]:
import subprocess

out = subprocess.Popen(['find', train_dir, '-name', "*\.[jJ][pP][gG]"], stdout = subprocess.PIPE).communicate()
train_num = out[0].count('\n')

out = subprocess.Popen(['find', valid_dir, '-name', "*\.[jJ][pP][gG]"], stdout = subprocess.PIPE).communicate()
valid_num = out[0].count('\n')

out = subprocess.Popen(['find', test_dir, '-name', "*\.[jJ][pP][gG]"], stdout = subprocess.PIPE).communicate()
test_num = out[0].count('\n')

print("Sanity check: " + str(train_num) + " train, " + str(valid_num) + " valid, " + str(test_num) + " test")

Sanity check: 14572 train, 1912 valid, 1933 test


In [5]:
def create_model(learning_rate, beta1, beta2, epsilon, dropout, decay):
    # Conv -> ReLu -> Pooling
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Conv -> ReLu -> Pooling
    model.add(keras.layers.Conv2D(32, (3, 3)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Conv -> ReLu -> Pooling
    model.add(keras.layers.Conv2D(64, (3, 3)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(keras.layers.Flatten())
    
    # Dense -> ReLu -> Dropout
    model.add(keras.layers.Dense(1024))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dropout(dropout))

    # Dense -> ReLu -> Dropout
    model.add(keras.layers.Dense(64))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dropout(dropout))

    # Dense
    model.add(keras.layers.Dense(18, activation='softmax'))

    opt = keras.optimizers.Adam(lr=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [38]:
import numpy as np
import os
import pickle

# Simulation parameters
epochs = 1
num_iters = 3

# Training parameters
learning_rates = [-6, -2]
betas_1  = [0.9, 0.999]
betas_2  = [0.9, 0.999]
epsilons = [-9, -7]
dropouts  = 0.8
decays = 0.0

# History parameters
best_acc = -1
best_params = ()
history = dict()

for _ in range(num_iters):
    
    ##### RANDOM HYPERPARAMETERS #####
    
    learning_rate = 10**np.random.uniform(learning_rates[0], \
                                          learning_rates[1])
    beta_1  = np.random.uniform(betas_1[0], betas_1[1])
    beta_2  = np.random.uniform(betas_2[0], betas_2[1])
    epsilon = 10**np.random.uniform(epsilons[0], \
                                    epsilons[1])
    dropout = np.random.uniform(dropouts)
    decay   = np.random.uniform(decays)
    
    params = (learning_rate, beta_1, beta_2, epsilon, dropout, decay)
    
    ##### CREATING MODEL #####
    
    model = create_model(learning_rate, beta_1, beta_2, epsilon, dropout, decay)

    ##### TRAINING MODEL #####
    
    model_name = "custom"
    weight_dir = "../model/%s"%model_name
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)

    weight_path = os.path.join(weight_dir, "temp_weights_%s.h5"%model_name)
    best_weight_path = os.path.join(weight_dir, "best_weights_%s.h5"%model_name)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True)
    stopper = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
    
    model.fit_generator(
        train_gen,
        steps_per_epoch=train_num // batch_size,
        epochs=epochs,
        validation_data=valid_gen,
        validation_steps=valid_num // batch_size,
        callbacks=[checkpointer,stopper])
    
    model.load_weights(weight_path)
    loss, acc = model.evaluate_generator(
        test_gen,
        steps = test_num // batch_size)
    print("Test accuracy： " + str(acc))
    
    if acc>best_acc:
        print("Saving as best accuracy")
        model.save_weights(best_weight_path)
        best_params = params
        best_acc = acc
        
    history[acc] = (loss, acc, params)
    del model
    
pickle.dump(history, open(os.path.join(weight_dir,'history_%s.p'%model_name),'w'))

Epoch 1/1
910/910 [==============================] - 142s - loss: 3.6568 - acc: 0.0582 - val_loss: 2.8897 - val_acc: 0.0475
Test accuracy： 0.0516431924883
Saving as best accuracy
Epoch 1/1
910/910 [==============================] - 145s - loss: 2.9118 - acc: 0.0622 - val_loss: 2.8885 - val_acc: 0.0522
Test accuracy： 0.0631194574857
Saving as best accuracy
Epoch 1/1
910/910 [==============================] - 142s - loss: 2.9056 - acc: 0.0653 - val_loss: 2.8865 - val_acc: 0.0675
Test accuracy： 0.0652060511215
Saving as best accuracy


In [40]:
learning_rate, beta_1, beta_2, epsilon, dropout, decay = best_params

model = create_model(learning_rate, beta_1, beta_2, epsilon, dropout, decay)

model.load_weights(best_weight_path)

loss, acc = model.evaluate_generator(
    test_gen,
    steps = test_num // batch_size)

print("Best accuracy： " + str(acc))

Best accuracy： 0.0678142931858


In [41]:
print(best_acc)

0.0652060511215
